In [1]:
from utils.command_handler import ScriptParser, Command

In [2]:
scr = ScriptParser()
lines = scr.load_script(scr.example_script_path)
scr.parse(lines)

In [3]:
def print_command(cmd, nest = 0):
    if type(cmd) == Command:
        print(f"{"   "*nest}{cmd.get_format()}")
    elif type(cmd) == list:
        for c in cmd:
            print_command(c, nest=nest+1)

In [4]:
print_command(scr.commands)

   laser_on()
   set_laser_duty(12)
   operator(var_a, =, 12)
      if(var_a, ==, 12)
      set_laser_duty(5)
   operator(var_x, =, 0)
      loop(4)
      laser_on()
      wait(2)
         if(var_a, >, 12)
         set_laser_duty(1)
      laser_off()
      operator(var_x, +=, 1)
      restart_block()


In [12]:
scr.commands